# Trabalho 1 
## Problema do caixeiro viajante utilizando algoritmo genético
---
**Alunos:**  
Leonardo Nogueira Agnelo - **RA:** 779801  
Matheus Paladini Pedrinho - **RA:** 769111

Importando bibliotecas

In [160]:
import numpy as np
import random

Insere o número de cidades e preenche a matriz de distância aleatóriamente 

In [161]:
#número de cidades
n_cidades = 6
#define uma distancia maxima entre duas cidades
dist_max_entre_2 = 100
#inicializa uma matriz zerada
matriz_dist = np.zeros((n_cidades,n_cidades), dtype = int)

#preenche a matriz com numero aleatórios entre 0 e no número maximo pre definido
for i in range(n_cidades):
  for j in range(i):
    if(i==j):
      matriz_dist[i][j] = 0
    else:
      matriz_dist[i][j] = random.randint(0,dist_max_entre_2)
      #faz com que a matriz seja reflexiva
      matriz_dist[j][i] = matriz_dist[i][j]

#mostra a distancia na matriz criada
matriz_dist

array([[ 0, 49,  9, 32, 66, 19],
       [49,  0,  8, 72, 64, 80],
       [ 9,  8,  0,  1, 88, 42],
       [32, 72,  1,  0, 60, 55],
       [66, 64, 88, 60,  0, 37],
       [19, 80, 42, 55, 37,  0]])

Define o número de individuos(possiveis caminhos) da população inicial e preenche aleatóriamente cada individuo com caminhos representados pelo numeor de cada, sem os valores da cidade final e inicial dado que ele ja é conhecido, ou seja 0.

In [162]:
#Número de individuos de uma população(deve ser par)
n_populacao = 10
#cria um array vetores preenchido com zeros
populacao = np.zeros((n_populacao,n_cidades-1), dtype=int)

#preenche o array de vetores onde cada vetor possui um caminho que passa por todas as cidades(menos a cidade 0)
for i in range(n_populacao):
  populacao[i] = random.sample(range(1,n_cidades),5)

#mostra a população criada
populacao

array([[1, 4, 5, 3, 2],
       [5, 3, 2, 4, 1],
       [1, 4, 3, 2, 5],
       [4, 2, 3, 1, 5],
       [5, 1, 3, 2, 4],
       [5, 4, 3, 1, 2],
       [3, 5, 4, 2, 1],
       [5, 4, 2, 3, 1],
       [3, 4, 2, 1, 5],
       [1, 2, 3, 5, 4]])

**Calcula a função fitness(objetivo)**  
De acordo com o problema apresentado a função objetivo retorna a soma das distancias de um caminho, em que quanto menor a distancia melhor o resultado


In [163]:
#função fitness onde é passado um caminho ocmo parametro
def fitness(caminho):
  fit = 0
  #referente a distância da cidade inicial até a primeira cidade do vetor
  inicio = matriz_dist[0][caminho[0]]
  #soma a distancia "inicio"
  fit += inicio

  #percorre a lista calculando a distância de uma cidade a outra
  for i in range(len(caminho)-1):
    #passo é a distância entre uma cidade e outra
    passo = matriz_dist[caminho[i]][caminho[i+1]]
    #soma a distância de um passo
    fit += passo
  
  #distância da última cidade da lista até a cidade 0(final)
  fim = matriz_dist[caminho[len(caminho)-1]][0]
  #soma a distância final
  fit += fim
  
  #retorna o valor total percorrido = fit
  return fit


#retorna uma lista de função fitness de cada individuo da população
def retorna_fitness(populacao):
  list_fit = []
  #chama a função fitness para cada individuo(caminho)
  for i in range(n_populacao):
    linha = fitness(populacao[i])
    list_fit.append(linha)

  return list_fit

Função Elitismo

In [164]:
#Retorna os o index dos dois melhores(menores) valores da lista de fitness
def elitismo(list_fitness):
  id1 = list_fitness.index(min(list_fitness))
  list_fitness.pop(id1)
  id2 = list_fitness.index(min(list_fitness))

  return id1, id2


Função seleção por torneio

In [165]:
#duas disputa de entre individuos aleatório que retorna os melhor segundo a função fitness
def torneio(list_fitness):

  cam1 = 0
  cam2 = 0

  #evita que os 2 indivios(caminos) sejam iguais
  while cam1 == cam2:
    ##[1]
    #escolhe 2 index aleatórios para disputarem entre si
    num = random.sample(range(1,n_populacao),2)
    #derifica qual o melhor indice
    if list_fitness[num[0]] < list_fitness[1]:
      cam1 = num[0]
    else:
      cam1 = num[1]

    #igual ##[1]
    num = random.sample(range(1,n_populacao),2)
    if list_fitness[num[0]] < list_fitness[1]:
      cam2 = num[0]
    else:
      cam2 = num[1]

    return cam1, cam2
  

Função de Crossover

In [166]:
#cruza dois individuos(caminhos) e geram dois individuos diferentes
def crossover(ids, pop):
  #como o tamanho do individuo é 5 posições é definido aleatóriamente se a metade vai ser na posição 2 ou 3
  corte = random.randint(2,3)

  #recebe os pais de acordo com os IDs passados
  pai1 = pop[ids[0]]
  pai2 = pop[ids[1]]

  #gera os filhos com o cruzamente dos pais
  filho1 = np.concatenate([pai1[:corte],pai2[corte:]])
  filho2 = np.concatenate([pai2[:corte],pai1[corte:]])

  #evita que as cidades após o cruzamente estejam duplicadas
  mata_dupla(filho1)
  mata_dupla(filho2)

  return filho1,filho2

#função que evita cidades duplicadas
def mata_dupla(filho):
  #vetor auxilliar que mostra quais cidade estão duplicadas
  aux = [0,0,0,0,0]
  #vetor auxiliar que mostra a posição do individuo que esta duplicada
  posi = []
  aux_posi = 0 

  #varre o individuo verificando onde há duplas
  for i in range(len(filho)):
    if aux[filho[i]-1]==0: #cidade ainda não apareceu no individuo
      aux[filho[i]-1]=1
    elif aux[filho[i]-1]==1: #cidade ja apareceu no individuo
      posi.append(i)

  #verifica quais cidades não apareceram no vetor e insere no posição que foi guardada no vetor auxiliar de posição
  for j in range(len(aux)):
    if(aux[j]==0): #cidade não aparece 
      filho[posi[aux_posi]] = j+1 #insere na posição do que tem uma dupla
      aux[j] = 1 
      aux_posi +=1


Função de Mutação

In [167]:
#de acordo com uma taxa(entre 0 e 1) de possível mutação altera a posição de duas cidades de um indivíduo
def mutacao(filhos, taxa):
  posi1 = 0
  posi2 = 0
  for i in range(len(filhos)):
    #define a probabilidade de mutação
    if random.random() <= taxa:
      #evita que as posições para troca sejam as mesmas
      while posi1 == posi2:
      #escolhe quais posições serão trocadas
        posi1 = random.randint(0,n_cidades-2)
        posi2 = random.randint(0,n_cidades-2)
      
      aux = 0
      #troca as posições 
      aux = filhos[i][posi1]
      filhos[i][posi1] = filhos[i][posi2]
      filhos[i][posi2] = aux
  
  return filhos


Programa principal

In [168]:
#numero de gerações
n_geracoes = 50
i = 0
sem_melhor = 0
melhor_anterior = 0

#executa um numero maximo de repetições ou até o melhor fitness encontrado permanecer por 20 gerações
while (i in range(n_geracoes)) and (sem_melhor < 20):

  #declara nova população vazia
  nova_populacao = np.zeros((n_populacao,n_cidades-1), dtype=int)

  #calcula o fitness da população(menor distancia)
  fit = retorna_fitness(populacao)
  id_melhor = fit.index(min(fit))

  #verifica quantas rodadas está com o mesmo fitness
  if(fit[melhor_anterior] == fit[id_melhor]):
    sem_melhor +=1
  else:
    sem_melhor = 0
  melhor_anterior = id_melhor

  #imprime qual a geração atual, o melhro caminho e o fitness dele
  print(" geração: ", i ,", melhor caminho : ",populacao[id_melhor],", fitness", fit[id_melhor])
  print()

  #Elitismo(mantem os 2 melhores camilhos para a proxima geração)
  elites = elitismo(fit.copy())
  nova_populacao[0] = populacao[elites[0]]
  nova_populacao[1] = populacao[elites[1]]
  n_filhos = 2

  #permanece no loop até que uma nova geração seja criada por filhos
  while n_filhos < n_populacao:
    #seleção por torneio(2 filhos)
    vencedores = torneio(fit)
    #cruzamento
    filhos = crossover(vencedores,populacao)
    #mutacao 50% de chance
    filhos = mutacao(filhos,1)
    #add filhos a populacao
    nova_populacao[n_filhos] = filhos[0]
    nova_populacao[n_filhos+1] = filhos[1]

    n_filhos +=2

  #substitui a populaçõa antiga pela nova
  populacao = nova_populacao.copy()
  i +=1

 geração:  0 , melhor caminho :  [5 4 3 1 2] , fitness 205

 geração:  1 , melhor caminho :  [5 4 3 1 2] , fitness 205

 geração:  2 , melhor caminho :  [2 3 1 4 5] , fitness 202

 geração:  3 , melhor caminho :  [2 3 1 4 5] , fitness 202

 geração:  4 , melhor caminho :  [1 2 3 4 5] , fitness 174

 geração:  5 , melhor caminho :  [1 2 3 4 5] , fitness 174

 geração:  6 , melhor caminho :  [1 2 3 4 5] , fitness 174

 geração:  7 , melhor caminho :  [1 2 3 4 5] , fitness 174

 geração:  8 , melhor caminho :  [1 2 3 4 5] , fitness 174

 geração:  9 , melhor caminho :  [1 2 3 4 5] , fitness 174

 geração:  10 , melhor caminho :  [1 2 3 4 5] , fitness 174

 geração:  11 , melhor caminho :  [1 2 3 4 5] , fitness 174

 geração:  12 , melhor caminho :  [1 2 3 4 5] , fitness 174

 geração:  13 , melhor caminho :  [1 2 3 4 5] , fitness 174

 geração:  14 , melhor caminho :  [1 2 3 4 5] , fitness 174

 geração:  15 , melhor caminho :  [1 2 3 4 5] , fitness 174

 geração:  16 , melhor caminho :  